In [40]:
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import Perceptron
from sklearn.model_selection import train_test_split
import sklearn.metrics as metric
import numpy as np
import pandas as pd

In [55]:
def readFramesFromCSV(filename):
    fullV5 = pd.read_csv(filename, usecols=[2]) # extraction of V5 column
    fullV5 = (fullV5 - 1024 )/ 200 # normalization
    frameLength = 2/(1/360) # 2 seconds => 2 * (1/360) samples per segment ..
    framesCount = int(fullV5.shape[0] / frameLength) # how many frames I can Extract ?
    usedV5 = fullV5[0:int(framesCount * frameLength)] # only useful last will be ignored
    frames = []
    for i in range(1,framesCount+1):
        frames.append(usedV5[int((i-1)*frameLength):int(i*frameLength)])
    return frames

In [57]:
# reading data from csv files into lists of dataframes
ListNormalFrames1 = readFramesFromCSV("100normal-pre-training.csv")
ListNormalFrames2 = readFramesFromCSV("101normal-pre-testing.csv")
ListRbbbFrames1 = readFramesFromCSV("118rbbb-pre-training.csv")
ListRbbbFrames2 = readFramesFromCSV("231rbbb-pre-testing.csv")

# conversion to matrix
Normal1 = np.array([np.array(xi) for xi in ListNormalFrames1])
Normal2 = np.array([np.array(xi) for xi in ListNormalFrames2])
Rbbb1 = np.array([np.array(xi) for xi in ListRbbbFrames1])
Rbbb2 = np.array([np.array(xi) for xi in ListRbbbFrames2])

# aggregation
Normalinput = np.concatenate((Normal1,Normal2))
Rbbbinput = np.concatenate((Rbbb1,Rbbb2))
Input = np.concatenate((Normalinput,Rbbbinput))
Normaloutput = np.ones((len(Normalinput),1))
Rbbboutput = np.zeros((len(Rbbbinput),1))
for i in range(len(Rbbboutput)):
    Rbbboutput[i] += -1
Output = np.concatenate((Normaloutput,Rbbboutput))

In [43]:
# Train set/ test set splitting
X_train, X_test, Y_train, Y_test = train_test_split(Input, Output, test_size=0.2, random_state=42)

In [44]:
# Reshaping
X_train = X_train.reshape((len(X_train),len(X_train[0])))
X_test = X_test.reshape((len(X_test),len(X_test[0])))
Y_train = Y_train.reshape(-1)
Y_test  = Y_test.reshape(-1)

In [52]:
# Perceptron training & prediction
ptn = Perceptron(max_iter=500,tol=1e-3 )
ptn.fit(X_train, Y_train)  
Y_pred = ptn.predict(X_test)
accuracy = metric.accuracy_score(Y_test.T, Y_pred, normalize=True)
print('acuracy =',accuracy)                         # show accracy score

acuracy = 0.7063711911357341


In [53]:
# MLP training & prediction
mlp = MLPClassifier(solver='lbfgs', hidden_layer_sizes=(8,8), activation='relu') # set the method
mlp.fit(X_train, Y_train)                    # training
Y_pred = mlp.predict(X_test)                      # prediction
accuracy = metric.accuracy_score(np.array(Y_test).flatten(), np.array(Y_pred).flatten(), normalize=True)
print('acuracy=',accuracy)        # show accuracy score

acuracy= 0.9542936288088643
